# Tutorial n°1 - SDK : Reachy's awakening

In this tutorial, we will learn how do a task with Reachy using the SDK client. 

Here, we are going to make Reachy do the awake sequence, which is a series of movements that makes it look like it is waking up. It involves moving its head and arms and can be used as a starting point for more complex sequences.

What you will learn:
- How to make it move its head
- How to make it move its arms
- How to synchronize head and arms movements


## 1. Prerequisites

To use the SDK client, you first need to install *reachy2-sdk*. If it is not the case, checkout the section below 👇
<details>

<summary>Install the Python library reachy2-sdk</summary>

In general, you'd better **work in a virtual environment**. You have 2 different ways to install the sdk : 
- by running the following command:

<code>
pip install reachy2-sdk -e .
</code>

- from source by following the instructions on the [GitHub repository](https://github.com/pollen-robotics/reachy2-sdk)

</details>

If you have never used the SDK client before, don't forget to do the [Getting Started notebooks](https://github.com/pollen-robotics/reachy2-sdk/tree/develop/src/examples), that will help you understand all the basics you need to know about it ! And if you want to go further, you can also check the [SDK documentation](https://pollen-robotics.github.io/reachy2-sdk/reachy2_sdk.html). 

## 2. Setup

### 2.1. Material

For this tutorial, you'll only need Reachy and that's it ! 

Generally speaking, before making the real robot move, working on a **virtual Reachy is strongly recommended** to visualize the movements and try to modify them. For that, you can follow the documentation [ajout du lien] and instanciate a ReachySDK with the *IP = 'localhost'*. Once you're sure that all your moves are safe for the robot, you can work on the real Reachy. 

### 2.2. Scene

> Your Reachy must be at a sufficient **height** so that its outstretched arms do not touch the mobile base. 

Put Reachy in a safe environment with enough place to move around, no one in reachable space and no obstacles (as there are no safety yet preventing the robot from colliding with its environment). And always keep the emergency stop button nearby! 



## 3. Preview

In this tutorial, we'll build step by step the program that will enable Reachy to look like it's awakening. Here, you can have a look at what Reachy will do at the end :

<p align="center">
  <img src="images/gif_awake.gif" alt="Gif 1" width="30%">
</p>

Now that you are all set, let's dig into it ! 

## 4. Let's build it !

### 4.1. Instanciation of the SDK

First, we will connect to the robot. To do so, we need to import the SDK client package and to instanciate a ReachySDK.

Two requirements :
- Your computer needs to be on the same network as the robot.
- You need to know your Reachy's IP : to do so, you have two options : 
    - you can check it on the dashboard (*.local_IP_adress_of_your_robot:8000*), section Network. 
    - you can have a look at the small screen on Reachy's back, that will show you one at a time its Ethernet and its Wifi IP. 

Now, let's connect to it.

In [ ]:
#import the package
from reachy2_sdk import ReachySDK

#connect to the robot
reachy = ReachySDK(host="localhost")  # replace 'localhost' with the actual IP address of your Reachy
print("Reachy is connected :", reachy.is_connected())

If you are getting the message "Could not connect to Reachy", make sure that :
-  Reachy is turned on
- the reachy2_core.service is running 
- the IP address is correct. 

*More info on the debug section of the [sdk documentation](https://pollen-robotics.github.io/reachy2-docs/help/help/recovering/).<br>*

### 4.2. Set Reachy ready

Reachy has its motors off by default. So first, we need to turn them on. 

In [ ]:
reachy.turn_on()
print(f"Reachy's motors are on : {reachy.is_on()}")

Then, we are going to place the robot in a neutral position, with the head looking straight ahead and the arms alongside its torso. You can use the *goto_posture* method to do so. 

In [ ]:
reachy.goto_posture("default")

Is your Reachy looking like this?

<p align="center">
  <img width=200 src="images/default_position.png" alt="Default position of Reachy" />
</p>

Great! Let's start implementing the awake sequence!

*If not, you may need to restart the core of your robot and go back from the beginning*.

### 4.3. Build the sequence

The awake sequence is composed of four parts:
1. Reachy "sleeps", i.e. the arms are swinging alongside its torso and the head is nodding slowly
2. Reachy looks like it is waking up, lifting its head and moving it from side to side
3. Reachy lifts its arms, moves them one by one and follows with its head the effector of the moving arm
4. Reachy nods and gets back to its default position

The easiest way to implement it (and it is a general rule when implementing complex behavior on the robot) is to break it down into smaller parts and implement them one by one independently. Once each part is working, you can combine them to create the full sequence.

Let's start with the first part: making Reachy "sleep".

#### 4.3.1. Asleep part

During the asleep sequence, the head of Reachy moves slowly up and down and the arms are outstretched, swinging back and forth. We will implement this sequence first.

##### Head movement

We will use the *look_at* method to make Reachy's head move up and down. The *look_at* method makes Reachy's head look at a specific point in the robot's frame. We will make Reachy's head look at a point that is above its current position and then below its current position. This will make Reachy's head move up and down.

*If you want more details about the mentioned methods, feel free to check the [documentation](https://pollen-robotics.github.io/reachy2-sdk/reachy2_sdk/parts/head.html) !*

Let's define a function that makes Reachy's head move up and down. We will call this function `asleep_head`.

In [ ]:
def asleep_head(reachy: ReachySDK):
    for _ in range(2):
        reachy.head.look_at(x=0.50, y=0.0, z=-0.20, duration=2.0, wait=False)
        reachy.head.look_at(x=0.50, y=0.0, z=-0.30, duration=2.0, wait=False)

print("Function asleep_head defined.")

Notice that the head will move up and down in a short range. If you want to make the head move in a larger range, just change the value of the z-coordinate in the look_at method.

Here, the parameter *wait* is set to False : we will not wait for the end of the movements in this part of the sequence as we want Reachy to move its head and arms simultaneously later.

Now, you can try it out and check Reachy's head!


In [ ]:
asleep_head(reachy)

##### Arm movement

There are different methods available to make Reachy's arms move: *goto* (with the list of joint positions or the goal pose of the effector), *translate_by* and *rotate_by*. Depending on the movement you want to achieve, you will prefer one over the others.

There, we want Reachy's arms to swing back and forth, standing straight on each side of the torso. 

The simplest way to do it is to place the arms in the default posture, get the current positions of the arms' joints and change the articular value of the shoulder pitch joint, by using the *goto* method. As you learned in the [SDK example](https://github.com/pollen-robotics/reachy2-sdk/blob/develop/src/examples/3_arm_and_gripper.ipynb), the list of joints is as follow : [shoulder pitch, shoulder roll, elbow yaw, elbow pitch, wrist roll, wrist pitch, wrist yaw], so we will change the first value of the list.

Let's define a function that makes Reachy's arms swing back and forth. We will call this function `asleep_arms`.

In [ ]:
def asleep_arms(reachy: ReachySDK):
    #first, define goal pitch positions for the left arm
    jpl = reachy.l_arm.get_current_positions() # Considering the arms are in the default position
    jpl_front = jpl.copy()
    jpl_front[0] += 6.0
    jpl_back = jpl.copy()
    jpl_back[0] -= 6.0

    #then, for the right arm
    jpr = reachy.r_arm.get_current_positions()
    jpr_front = jpr.copy()
    jpr_front[0] += 6.0
    jpr_back = jpr.copy()
    jpr_back[0] -= 6.0

    for _ in range(2):
        # Move left arm backward and right arm forward first and then vice versa
        reachy.l_arm.goto(jpl_back, duration=2.0, wait=False)
        reachy.l_arm.goto(jpl_front, duration=2.0, wait=False)

        reachy.r_arm.goto(jpr_front, duration=2.0, wait=False)
        reachy.r_arm.goto(jpr_back, duration=2.0, wait=False)

    #go back to the default posture for both arms
    reachy.l_arm.goto_posture("default", wait=False)
    reachy.r_arm.goto_posture("default", wait=True)

print("Function asleep_arms defined.")

Now, try the function on Reachy and check its arms!

In [ ]:
reachy.goto_posture("default", wait=True)

asleep_arms(reachy)

Notice that the arms swing back and forth in a desynchronized way. When the left arm is moving forward, the right arm is moving backward.

For the range of the arm movement, we chose a range of 12 degrees for the shoulder pitch joint. If you want to change the range of the movement, just change the value 6.0 in the fonction.

##### Combine the head and arm movements

Now that we have implemented both arms and head movements separately and validate them, we can combine them to make the full asleep behavior in a single function `asleep`.

In [ ]:
def asleep(reachy: ReachySDK):
    asleep_head(reachy)
    asleep_arms(reachy)

print("Function asleep defined.")

Let's try the function! You should see both sequence for the head and the arms running simultaneously.

In [ ]:
reachy.goto_posture("default", wait=True)

asleep(reachy)

> 💡 One important thing to notice is that in order to make the movements of the head and arms synchronized, we need NOT to wait for the end of the movements of the head when we start the movements of the arms (hence the *wait=False* argument in the *look_at* calls). 
> Same goes for the arms, if we wait for the end of the movement, let's say of the left arm before starting the right arm, the movements will be desynchronized.
Calling the *look_at* and *goto* methods with *wait=False* will load the movements in the goto queue of the SDK which will run them one by one sequentially for each part (left arm, right arm and head).

Try to change some of the *wait* arguments to True, to check how it is impacting the movements !

Because the duration of the movement of the head and of each arm is the same (4 seconds), we only need to wait for the end of the last movement to be sure that the whole asleep behavior is finished (here we arbitrarily chose to wait for the end of the right arm movement).

That's it for the asleep part! We can now move on to the next part of the awake sequence: making Reachy looks like it is waking up.

#### 4.3.2. Waking up part

Now, we want Reachy to lift its head and move it from side to side. 

We are going to define a single function called `head_awake_sequence` to set all the sequence. We can use *look_at* method, with long duration moves to make it look like it's waking up slowly.

In [ ]:
def head_awake_sequence(reachy: ReachySDK):
    # Lift the head up slowly (it is supposed to be just waking up)
    reachy.head.look_at(x=0.5, y=0.0, z=0.0, duration=4.0, wait=False)

    # Look left and right
    reachy.head.look_at(x=0.5, y=-0.2, z=-0.2, duration=2.0, wait=False)
    reachy.head.look_at(x=0.5, y=0.0, z=-0.05, duration=2.0, wait=False)
    reachy.head.look_at(x=0.5, y=0.2, z=-0.2, duration=2.0, wait=False)

    # Look straight ahead again
    reachy.head.look_at(x=0.5, y=0.0, z=0.0, duration=2.0, wait=True)

print("Function head_awake_sequence defined.")

You can now try it on the robot ! 

> 💡 Note that we only wait for the last *look_at* to be finished to consider the sequence done. However in this case, since only head movements are composing the awake sequence and are performed one after the other, we could have set wait=True for every *look_at* call, without making any difference. 

In [ ]:
head_awake_sequence(reachy)

Perfect ! Now, we can move on to the next step. 

#### 4.3.3. Arms swinging and end effector tracking

Now, we want Reachy to lift its arms, move them one by one and follow the effector of the moving arm with its head.

This part is a bit more complex as it involves moving the arms and the head simultaneously and making the robot track itself. 

But don't worry, we are going to break down how we can implement this behavior:


1. **Lift the arms and move them one by one**: here we want to place the arms at 90 degree angle and move them one by one forward and backward. 
- To lift the arms at 90 degree angle, we will just call the *goto_posture* method which has it as a predefined posture available.
- To move the arm, we can use the *translate_by* method : it allows us not to worry about the joints positions needed for this nor getting the appropriate pose matrices for a *goto* call.  We only need to specify the offset from the current pose in the robot's frame that we want to apply to the effector of the arm. </br>


2. **Make the head follow the moving arm effector** : as before, we will use the *look_at* method which takes as argument the coordinates x, y, z of a point in the robot's frame. Good news is that we can easily get the coordinates of the effector center in the robot's frame by calling the *forward_kinematics* method of the arm. We will use it to get the coordinates of the moving arm effector and make the head look at this point.

And we can define a function `arm_swing_and_effector_tracking` to deal with this entire behavior.


In [ ]:
def arm_swing_and_effector_tracking(reachy: ReachySDK):
    # Put the arms at 90 degrees angle and wait for it to be done
    reachy.goto_posture("elbow_90", wait=True)

    for arm in [reachy.l_arm, reachy.r_arm]:
        x, y, z = arm.forward_kinematics()[:3, 3]
        # The first look_at has a longer duration to avoid a sudden head movement
        reachy.head.look_at(x=x, y=y, z=z, duration=1.5, wait=True)

        arm.translate_by(x=0.2, y=0.0, z=0.1, wait=False) 
        arm.translate_by(x=-0.2, y=0.0, z=-0.1, wait=False)
        arm.translate_by(x=0.2, y=0.0, z=0.1, wait=False)
        last_gotoid = arm.translate_by(x=-0.2, y=0.0, z=-0.1, wait=False)

        # while the arm is moving, the head is tracking the effector
        while not reachy.is_goto_finished(last_gotoid):
            x, y, z = arm.forward_kinematics()[:3, 3]
            reachy.head.look_at(x=x, y=y, z=z, duration=0.05, wait = True, interpolation_mode='linear')  # head tracking the effector at 20Hz

        reachy.head.look_at(x=0.5, y=0.0, z=0.0, duration=1.0, wait = True)

print("Function arm_swing_and_effector_tracking defined.")

> 💡 Note that we don't wait for the end of the movements of the moving arm in this function as we want the head to follow the effector while the arm is moving. We only wait for the end of the last movement to consider the whole sequence done. While the arm is moving, we call the forward_kinematics method of the arm to get the coordinates of the effector and make the head look at this point.

For the *look_at* calls of the head, we need to set *wait=True* otherwise a large number of *look_at* calls will be loaded in the goto queue of the sdk and the head will not be able to follow the effector of the moving arm and lots of delay will appear. <br>
Again, you can try to change the *wait* argument of the *look_at()*, to see the difference ! 

Now, let's try the function on the robot !

In [ ]:
arm_swing_and_effector_tracking(reachy)

That's working ! Now, we can move on to the last part of our sequence. 

#### 4.3.4. Nodding and getting back to the default posture

To finish, Reachy nods with its head and gets back to the default posture.

We will define a function `nod_and_default_posture` for this behavior.
To make the nodding part, we will use the *look_at()* method as before. To get back to the default position, we will use the *goto_posture()* method that we also used before in this project.

In [ ]:
def nod_and_default_posture(reachy):
    reachy.head.look_at(0.5, 0.0, -0.2, 1.0)
    reachy.head.look_at(0.5, 0.0, 0.0, 0.5, wait=True)
    reachy.l_arm.goto_posture("default")
    reachy.r_arm.goto_posture("default", wait=True)

print("Function nod_and_default_posture defined.")

Now, you can try the function on the robot :

In [ ]:
nod_and_default_posture(reachy)

Perfect ! So now we have all the building blocks we need to construct a function to link this entire sequence together. Let's do it !

### 4.4. Put it all together

Now that we have implemented each behavior separately, it's time to compose them to make the full awake sequence!

The sequence is just a composition of the different behaviors we implemented and each behavior will be executed sequentially. We will call the functions we defined in the right order to make Reachy do the awake sequence.

We will gather the different parts of the awake sequence in a single function `awake`.

In [ ]:
def awake(reachy: ReachySDK):
    asleep(reachy)
    head_awake_sequence(reachy)
    arm_swing_and_effector_tracking(reachy)
    nod_and_default_posture(reachy)

print("Function awake defined.")

Call the function to see Reachy doing the whole awake sequence!

In [ ]:
awake(reachy)

Well done, we did it, Reachy has woken up ! 

## 5. Final tips

Now, you've learned how to build a sequence on Reachy using only the SDK Client with: 
- head movements
- arms movements
- synchronization between Reachy's parts

You can now use this sequence as a starting point to create other complex behaviors on the robot. Feel free to modify the movements, the duration of the movements, the order of the movements, etc. to get more familiar with the SDK, the robot and to check whether you can make Reachy do what you want it to do ! 

Here are some general tips to keep in mind to be the best at implementing complex behaviors on Reachy:

- **Always test behavior** on a fake robot before running it on the real robot! This will help you check if the behavior is doing what you expect it to do and to avoid any potential damage to the robot.
- If you are working on the real robot, make sure that it has enough space around it to move its arms and head. Especially, make sure that the arms will not be blocked by objects such as a table as there are no safety yet preventing the robot for colliding with its environment.
- Split the behavior you wish to develop into smaller parts and implement them one by one. Once each part is working, you can combine them to create the full sequence. Go slow and test each part before moving on to the next one. 


## 6. Skip to the next tutorial ! 

Here, we've covered just a few of the methods that can be used on Reachy. To discover more ways of controlling the robot, don't hesitate to continue following the tutorials ! 

**1. Reachy's awakening (with only SDK)**

2. Reachy the mime (with only SDK)

3. Reachy the greengrocer (with SDK & Pollen-Vision)


You'll be soon an expert to control Reachy ! 